In [0]:
%sql
-- Create SQL Procedure for ADS_RDS_EVENT_STATUS_IDENTIF_STATE ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN'
AS
BEGIN

BEGIN

declare map_id = "ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING";
declare schema_name = 'gap_catalog.ads_owner';
declare dif_table = 'gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF';

declare max_key bigint default 1;
DECLARE ddl string DEFAULT "";

set max_key = (
  with max_df as(
  select
    max(CPPROP_KEY) as max_cpprop_key
  from
    gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
  )
  select
    bigint(max_cpprop_key)
  from max_df
);
truncate table gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING;

INSERT INTO gap_catalog.ads_etl_owner.XC_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING 
      SELECT
          STG_SMA.CPPROP_KEY,
          STG_SMA.CPPROP_SOURCE_ID,
          STG_SMA.CPPROP_SOURCE_SYSTEM_ID,
          STG_SMA.CPPROP_SOURCE_SYS_ORIGIN,
          STG_SMA.CPPROP_PARENT_KEY,
          -1 AS CASEPH_KEY,
          STG_SMA.CPPTP_KEY,
          NVL(CAS.CTP_KEY, -1) AS CTP_KEY,
          STG_SMA.CASE_KEY,
          NVL(CAS.CASE_START_DATE, TO_DATE('01011000', 'ddMMyyyy')) AS CASE_START_DATE,
          STG_SMA.CPPROP_VALUE_TEXT,
          STG_SMA.CPPROP_VALUE_DATE,
          STG_SMA.CPPROP_VALUE_NUMBER,
          -1 AS CPPRV_KEY
      FROM
          gap_catalog.ads_owner.CASE_PHASE_PROPERTIES STG_SMA
      LEFT JOIN
          gap_catalog.ads_owner.CASES CAS
      ON
          CAS.CASE_KEY = STG_SMA.CASE_KEY
          AND CAS.CTP_KEY = 1429
      WHERE
          STG_SMA.CTP_KEY = 1429
;

DROP TABLE IF EXISTS gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF;

set ddl = 'CREATE TABLE gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_MAIN_BUSY_BANKING_ADS_MAP_SCD_DIFF
( tech_del_flg  char(1),
  tech_new_rec  char(1),
  tech_rid      varchar(255),
  CPPROP_KEY  INTEGER,
  CPPROP_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH ' || max_key || ' INCREMENT BY 1),
  CPPROP_SOURCE_ID  varchar(400),
  CPPROP_SOURCE_SYSTEM_ID  varchar(120),
  CPPROP_SOURCE_SYS_ORIGIN  varchar(120),
  CPPROP_PARENT_KEY  INTEGER,
  CASEPH_KEY  INTEGER,
  CPPTP_KEY  INTEGER,
  CTP_KEY  INTEGER,
  CASE_KEY  INTEGER,
  CASE_START_DATE  DATE,
  CPPROP_VALUE_TEXT  varchar(4000),
  CPPROP_VALUE_DATE  DATE,
  CPPROP_VALUE_NUMBER  INTEGER,
  CPPRV_KEY  INTEGER)
;';

execute immediate ddl;

update  gap_catalog.ads_owner.CASE_PHASE_PROPERTIES set
       CPPROP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
       CPPROP_UPDATE_PROCESS_KEY = 13165092,
       CPPROP_CURRENT_FLAG = 'N', 
       CPPROP_VALID_TO = to_date('20250911','yyyyMMdd')-1
 where CPPROP_CURRENT_FLAG = 'Y'
   and CPPROP_VALID_TO = to_date('30000101','yyyyMMdd')
   and cpprop_key||'.'||cpprop_valid_from||'.'||ctp_key||'.'||case_start_date in (select tech_rid from identifier(dif_table) where tech_rid is not null);

insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
 ( CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   CPPROP_VALID_FROM, 
   CPPROP_VALID_TO,
   CPPROP_CURRENT_FLAG, 
   CPPROP_DELETED_FLAG, 
   CPPROP_INSERTED_DATETIME, 
   CPPROP_INSERT_PROCESS_KEY, 
   CPPROP_UPDATED_DATETIME, 
   CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY)
select CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
   to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
   'Y' as CPPROP_CURRENT_FLAG, 
   tech_del_flg as CPPROP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
   p_process_key as CPPROP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
   p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
  from identifier(dif_table)
 where tech_new_rec = 'N';

insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
 ( CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   CPPROP_VALID_FROM, 
   CPPROP_VALID_TO,
   CPPROP_CURRENT_FLAG, 
   CPPROP_DELETED_FLAG, 
   CPPROP_INSERTED_DATETIME, 
   CPPROP_INSERT_PROCESS_KEY, 
   CPPROP_UPDATED_DATETIME, 
   CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY)
select CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
   to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
   'Y' as CPPROP_CURRENT_FLAG, 
   tech_del_flg as CPPROP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
   p_process_key as CPPROP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
   p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
  from identifier(dif_table)
 where tech_new_rec = 'Y';


END

/*    -- Declare local variables
    DECLARE dif_table_name STRING DEFAULT 'gap_catalog.ads_etl_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF';
    DECLARE my_log_id STRING;
    BEGIN
        CALL gap_catalog.log.start_performance_test(
            test_case_name => 'ADS_SMA_PROC',
            step_name => 'ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN',
            psource_sys_origin  => 'MAIN',
            process_key => p_process_key,
            target_table => 'CASE_PHASE_PROPERTIES',
            log_id => my_log_id,
            p_load_date => p_load_date
        );

  truncate table gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_POT_OWN;

  INSERT INTO gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_POT_OWN
  SELECT
    NULL AS cpprop_key,
    src.caseph_source_id || '.POTENTIALOWNERS.SMA_MONITOR_EVENTS.' || (src.idx - 1) AS CPPROP_SOURCE_ID,
    'SMA' AS CPPROP_SOURCE_SYSTEM_ID,
    'SMA_MONITOR_EVENTS' AS CPPROP_SOURCE_SYS_ORIGIN,
    -1 AS CPPROP_PARENT_KEY,
    cp.CASEPH_KEY AS CASEPH_KEY,
    COALESCE(t.CPPTP_KEY, -1) AS CPPTP_KEY,
    COALESCE(c.CASE_KEY, -1) AS CASE_KEY,
    COALESCE(c.CTP_KEY, -1) AS CTP_KEY,
    COALESCE(c.CASE_START_DATE, TO_DATE('1000-01-01', 'yyyy-MM-dd')) AS CASE_START_DATE,
    src.value AS CPPROP_VALUE_TEXT,
    NULL AS CPPROP_VALUE_DATE,
    NULL AS CPPROP_VALUE_NUMBER,
    -1 AS CPPRV_KEY,
    'N' AS CPPROP_DELETED_FLAG
  FROM (
    SELECT
      cidla,
      caseph_source_id,
      idx,
      value
    FROM (
      SELECT
        cidla,
        timestamp,
        type,
        jsondata,
        get_json_object(jsondata, '$.humanTask.id') AS caseph_source_id,
        ROW_NUMBER() OVER (
          PARTITION BY cidla, get_json_object(jsondata, '$.humanTask.id')
          ORDER BY timestamp DESC
        ) AS rn,
        EXPLODE(
          FROM_JSON(
            get_json_object(jsondata, '$.humanTask.potentialOwners'),
            'array<string>'
          )
        ) AS value_with_idx
      FROM gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full
      WHERE (
        type LIKE 'HT_%'
        OR type = 'CASE_CHANGE_CASETYPE'
        OR type IN ('REQUISITION_CREATED', 'REQUISITION_COMPLETED')
      )
      AND  SYS_EFFECTIVE_DATE = CAST(p_load_date AS DATE)
      AND type <> 'HT_RECONCILE'
    ) base
    LATERAL VIEW posexplode(split(value_with_idx, ',')) AS idx, value
    WHERE rn = 1
  ) src
  LEFT JOIN gap_catalog.ads_owner.case_phases cp
    ON src.caseph_source_id = cp.caseph_source_id
    AND cp.cpcat_key = 2
    AND DATE(p_load_date) BETWEEN cp.CASEPH_VALID_FROM AND cp.CASEPH_VALID_TO
  LEFT JOIN gap_catalog.ads_owner.CASE_PHASE_PROPERTY_TYPES t
    ON t.CPPTP_SOURCE_ID = 'POTENTIALOWNERS'
    AND t.CPPTP_SOURCE_SYSTEM_ID = 'RDS'
    AND t.CPPTP_SOURCE_SYS_ORIGIN = 'RDS_ANALYTICALCPPROPERTYTYPES'
    AND t.CPPTP_VALID_TO = DATE '3000-01-01'
  LEFT JOIN gap_catalog.ads_owner.cases c
    ON c.case_source_id = src.cidla
    AND c.CASE_SOURCE_SYSTEM_ID = 'SMA'
    AND c.case_source_sys_origin = 'SMA_MONITOR_EVENTS'
    AND DATE(p_load_date) BETWEEN c.case_valid_from AND c.case_valid_to;

DROP TABLE IF EXISTS gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF;

create  table gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF
( tech_del_flg  char(1),
  tech_new_rec  char(1),
  tech_rid      varchar(255),
  CPPROP_KEY  BIGINT,
  CPPROP_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
  CPPROP_SOURCE_ID  varchar(400),
  CPPROP_SOURCE_SYSTEM_ID  varchar(120),
  CPPROP_SOURCE_SYS_ORIGIN  varchar(120),
  CPPROP_PARENT_KEY  BIGINT,
  CASEPH_KEY  BIGINT,
  CPPTP_KEY  BIGINT,
  CTP_KEY  BIGINT,
  CASE_KEY  BIGINT,
  CASE_START_DATE  DATE,
  CPPROP_VALUE_TEXT  varchar(4000),
  CPPROP_VALUE_DATE  DATE,
  CPPROP_VALUE_NUMBER  DOUBLE,
  CPPRV_KEY  BIGINT)
;

insert into gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF
(
  tech_del_flg,
  tech_new_rec,
  tech_rid,
  CPPROP_KEY,
  CPPROP_SOURCE_ID,
  CPPROP_SOURCE_SYSTEM_ID,
  CPPROP_SOURCE_SYS_ORIGIN,
  CPPROP_PARENT_KEY,
  CASEPH_KEY,
  CPPTP_KEY,
  CTP_KEY,
  CASE_KEY,
  CASE_START_DATE,
  CPPROP_VALUE_TEXT,
  CPPROP_VALUE_DATE,
  CPPROP_VALUE_NUMBER,
  CPPRV_KEY
)
SELECT
  NULL AS cpprop_key,
  src.caseph_source_id || '.POTENTIALOWNERS.SMA_MONITOR_EVENTS.' || (src.idx - 1) AS CPPROP_SOURCE_ID,
  'SMA' AS CPPROP_SOURCE_SYSTEM_ID,
  'SMA_MONITOR_EVENTS' AS CPPROP_SOURCE_SYS_ORIGIN,
  -1 AS CPPROP_PARENT_KEY,
  cp.CASEPH_KEY AS CASEPH_KEY,
  COALESCE(t.CPPTP_KEY, -1) AS CPPTP_KEY,
  COALESCE(c.CASE_KEY, -1) AS CASE_KEY,
  COALESCE(c.CTP_KEY, -1) AS CTP_KEY,
  COALESCE(c.CASE_START_DATE, TO_DATE('1000-01-01', 'yyyy-MM-dd')) AS CASE_START_DATE,
  src.value AS CPPROP_VALUE_TEXT,
  NULL AS CPPROP_VALUE_DATE,
  NULL AS CPPROP_VALUE_NUMBER,
  -1 AS CPPRV_KEY,
  'N' AS CPPROP_DELETED_FLAG
FROM (
  SELECT
    cidla,
    caseph_source_id,
    idx,
    value
  FROM (
    SELECT
      cidla,
      timestamp,
      type,
      jsondata,
      get_json_object(jsondata, '$.humanTask.id') AS caseph_source_id,
      ROW_NUMBER() OVER (
        PARTITION BY cidla, get_json_object(jsondata, '$.humanTask.id')
        ORDER BY timestamp DESC
      ) AS rn,
      EXPLODE(
        FROM_JSON(
          get_json_object(jsondata, '$.humanTask.potentialOwners'),
          'array<string>'
        )
      ) AS value_with_idx
    FROM gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full
    WHERE (
      type LIKE 'HT_%'
      OR type = 'CASE_CHANGE_CASETYPE'
      OR type IN ('REQUISITION_CREATED', 'REQUISITION_COMPLETED')
    )
    AND  SYS_EFFECTIVE_DATE =  CAST(p_load_date AS DATE)
    AND type <> 'HT_RECONCILE'
  ) base
  LATERAL VIEW posexplode(split(value_with_idx, ',')) AS idx, value
  WHERE rn = 1
) src
LEFT JOIN gap_catalog.ads_owner.case_phases cp
  ON src.caseph_source_id = cp.caseph_source_id
  AND cp.cpcat_key = 2
  AND p_load_date BETWEEN cp.CASEPH_VALID_FROM AND cp.CASEPH_VALID_TO
LEFT JOIN gap_catalog.ads_owner.CASE_PHASE_PROPERTY_TYPES t
  ON t.CPPTP_SOURCE_ID = 'POTENTIALOWNERS'
  AND t.CPPTP_SOURCE_SYSTEM_ID = 'RDS'
  AND t.CPPTP_SOURCE_SYS_ORIGIN = 'RDS_ANALYTICALCPPROPERTYTYPES'
  AND t.CPPTP_VALID_TO = DATE '3000-01-01'
LEFT JOIN gap_catalog.ads_owner.cases c
  ON c.case_source_id = src.cidla
  AND c.CASE_SOURCE_SYSTEM_ID = 'SMA'
  AND c.case_source_sys_origin = 'SMA_MONITOR_EVENTS'
  AND  CAST(p_load_date AS DATE) BETWEEN c.case_valid_from AND c.case_valid_to;

update
  gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
set
  CPPROP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
  CPPROP_UPDATE_PROCESS_KEY =p_process_key,
  CPPROP_CURRENT_FLAG = 'N',
  CPPROP_VALID_TO = CAST(p_load_date AS DATE) - 1
where
  CPPROP_CURRENT_FLAG = 'Y'
  and CPPROP_VALID_TO = to_date('30000101', 'yyyyMMdd')
  and cpprop_key || '.' || cpprop_valid_from || '.' || ctp_key || '.' || case_start_date in (
    select
      tech_rid
    from
      gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF
    where
      tech_rid is not null
  );

insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
(
   CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   CPPROP_VALID_FROM, 
   CPPROP_VALID_TO,
   CPPROP_CURRENT_FLAG, 
   CPPROP_DELETED_FLAG, 
   CPPROP_INSERTED_DATETIME, 
   CPPROP_INSERT_PROCESS_KEY, 
   CPPROP_UPDATED_DATETIME, 
   CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
)
select 
   CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
   to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
   'Y' as CPPROP_CURRENT_FLAG, 
   tech_del_flg as CPPROP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
   p_process_key as CPPROP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
   p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
from gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF
where tech_new_rec = 'N';

insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
(
   CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   CPPROP_VALID_FROM, 
   CPPROP_VALID_TO,
   CPPROP_CURRENT_FLAG, 
   CPPROP_DELETED_FLAG, 
   CPPROP_INSERTED_DATETIME, 
   CPPROP_INSERT_PROCESS_KEY, 
   CPPROP_UPDATED_DATETIME, 
   CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
)
select 
  xxhash64(uuid()) AS CPPROP_KEY, 
   CPPROP_SOURCE_ID, 
   CPPROP_SOURCE_SYSTEM_ID, 
   CPPROP_SOURCE_SYS_ORIGIN, 
   CPPROP_PARENT_KEY, 
   CASEPH_KEY, 
   CPPTP_KEY, 
   CTP_KEY, 
   CASE_KEY, 
   CASE_START_DATE, 
   CPPROP_VALUE_TEXT, 
   CPPROP_VALUE_DATE, 
   CPPROP_VALUE_NUMBER, 
   to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
   to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
   'Y' as CPPROP_CURRENT_FLAG, 
   tech_del_flg as CPPROP_DELETED_FLAG, 
   CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
   p_process_key as CPPROP_INSERT_PROCESS_KEY, 
   CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
   p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
   CPPRV_KEY
from gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_POT_OWN_ADS_MAP_SCD_DIFF
where tech_new_rec = 'Y';
      CALL gap_catalog.log.complete_performance_test(
            plog_id => my_log_id
        );

    END;
    
END;